In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import psycopg2
from sqlalchemy import create_engine
import config
import contextily as ctx
import imageio

### Read in positions

In [2]:
DATABASE_URL = config.DATABASE_URL
engine = create_engine(DATABASE_URL, echo=False)

In [3]:
db_table = 'bus_positions'
query = f'''SELECT * FROM rushhour2;'''
df = pd.read_sql(query, engine)
df = df.drop_duplicates()
df.shape

(108968, 12)

### Read in lines
Downloaded in GTFS static format

In [4]:
df_lines = pd.read_csv('data/gtfs_static/shapes.txt')

### Preprocess bus positions

In [5]:
def preprocessing_positions(df):
    '''Format bus positions into geodataframe format.
    -------------
    Parameter:
    - df: DataFrame
    Output:
    - GeoDataFrame
    '''
    # adjust time (from UTC to PST)
    df['update_datetime'] = pd.to_datetime(df['update_time'], unit = 's')
    df['update_datetime'] = df['update_datetime'] + pd.offsets.Hour(-7)
    df = df.set_index('update_datetime')
    df= df.sort_index()
    
    # sample to 1 position per minute
    df_sampled = pd.DataFrame()
    for trip in df.trip_id.unique():
        tmp = df[df['trip_id'] == trip].resample('60s').first()
        df_sampled = df_sampled.append(tmp)
    
    # convert into geodataframe
    gdf = gpd.GeoDataFrame(df_sampled, geometry=gpd.points_from_xy(df_sampled.longitude, df_sampled.latitude))
    
    # change projection to fit with background map
    gdf.crs = {'init' :'epsg:4326'}
    gdf = gdf.to_crs(epsg=3857)
    
    # sort index
    gdf = gdf.sort_index()
    
    return gdf

In [6]:
gdf_pos = preprocessing_positions(df)
gdf_pos.shape

(43318, 13)

### Preprocess bus lines

In [7]:
def preprocessing_lines(df):
    '''Format bus lines into geodataframe format.
    -------------
    Parameter:
    - df: DataFrame
    Output:
    - GeoDataFrame
    '''
    tmp = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.shape_pt_lon, df.shape_pt_lat))
    gdf = tmp.groupby(['shape_id'])['geometry'].apply(lambda x: LineString(x.tolist()))
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
    
    # change projection to fit with background map
    gdf.crs = {'init' :'epsg:4326'}
    gdf = gdf.to_crs(epsg=3857)
    
    return gdf

In [8]:
gdf_lines = preprocessing_lines(df_lines)
gdf_lines.shape

(1053, 1)

### Draw maps

In [9]:
def create_maps(xlim, ylim, path_images):
    '''Creates a map with bus positions per minute
    ------------
    Parameters:
    - xlim: list (with coordinates)
    - ylim: list (with coordinates)
    - path_images: string (filepath)
    ------------
    Output: None
    '''
    plt.style.use('fivethirtyeight')
    count = 0 
    for i in gdf_pos.index.unique():
        count += 1
        fig, ax = plt.subplots(figsize=(15, 15))

        ax.set_aspect(aspect='equal')
        ax.set_axis_off()
        ax.set_ylim(ylim)
        ax.set_xlim(xlim)

        gdf_lines.plot(ax=ax, linewidth=1, alpha=0.05, color='red', zorder=1)

        gdf_pos.loc[i].plot(ax=ax, alpha=0.5, linewidth=1, color='red', zorder=2)
        ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

        time = i.strftime("%H:%M")
        fig.text(0.1, 0.25, i.strftime(time), fontsize=100, color='gray', alpha=0.2, weight='bold')

        plt.savefig(f'{path_images}/{i.strftime("%H%M")}.png')
        plt.close()

In [10]:
# Set coordinates
ylim_greater_portland = [5665000, 5725000]
xlim_greater_portland = [-13707000, -13615000]
ylim_downtown = [5698000, 5708000]
xlim_downtown = [-13660000, -13650000]
path_images_greater_portland = 'data/greater_portland'
path_images_downtown = 'data/downtown'

In [11]:
create_maps(xlim_greater_portland, ylim_greater_portland, path_images_greater_portland)

In [12]:
create_maps(xlim_downtown, ylim_downtown, path_images_downtown)

### Create GIF

In [13]:
def create_gif(path_images, path_gif):
    '''Creates gif from maps
    ------------
    Parameters:
    - path_images: string (filepath)
    - path_gif: string (filepath & name)
    ------------
    Output: None
    '''
    list_of_images = []

    for i in gdf_pos.index.unique():
        img = imageio.imread(f'{path_images}/{i.strftime("%H%M")}.png')
        list_of_images.append(img)

    imageio.mimsave(path_gif, list_of_images, fps=10)

In [14]:
create_gif(path_images_downtown, 'data/greater_portland.gif')

In [15]:
create_gif(path_images_downtown, 'data/downtown.gif')